In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime,date
import re
import json
from pandas.api.types import is_numeric_dtype

# from simple_salesforce import Salesforce, format_soql, format_external_id
# from simple_salesforce.exceptions import SalesforceMalformedRequest
# import simpler_sf
from sqlalchemy import create_engine
#import pymysql

# from mysql import connector


# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [ ]:
# usuario = 'siempreread'
# contrasena = 'S13mpr3_Ch1l3'
# url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "MetLife_A"
plugin_autenticacion = "mysql_native_password"

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

#table.to_sql(name='rut_unico_tiempo',con=motor_mysql_mariadb,if_exists = 'append')


# archivo de salida modificar segun archivo
# archivo = 'C:/Users/jmartinez/Desktop/casos nuevos/clientes/1/casos_faltantes_segun_archivo.xlsx'

In [ ]:
# table.to_sql(name='prueba',con=motor_mysql_mariadb,if_exists = 'append')
rut_unicos = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unicos"
rut_unicos = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("-", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("–", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(" ", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.upper()
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.strip()

rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("-", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("–", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(" ", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.upper()
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.strip()

rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(",", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(".", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("-", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┐", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("═", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("╔", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┌", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.title()
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.strip()

In [79]:
mes = 1
año = "2023"
fecha = año + "-" + str(mes) + "-" + "01"  
mesesDic = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

str(mesesDic[mes])



sftp = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/sftp.xlsx"


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"


derty = pd.read_excel(sftp)

In [80]:
derty.shape

(387665, 10)

In [81]:
metlife_sftp = pd.DataFrame()
metlife_sftp = derty.copy()

In [82]:


metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("-", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("–", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(" ", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.upper()
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.strip()

metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("-", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("–", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(" ", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.upper()
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.strip()

metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(",", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(".", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("-", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┐", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("═", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("╔", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┌", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.title()
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.strip()

metlife_sftp["poliza"] = metlife_sftp["poliza"].astype(str)
metlife_sftp["poliza"] = metlife_sftp["poliza"].str.strip()



#*******************sacar primer cero
metlife_sftp.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\3662042864.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\3662042864.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\3662042864.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["Nombre"] = metlife_sftp["Nomb

(387665, 10)

In [83]:
def siPrimerCero(y):    
    if len(y) <= 4:
        y =  "NONE"
    if y[0] == "0":
        y = y[1:]
    return str(y)
    

In [84]:
metlife_sftp["rut3"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut4"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp = metlife_sftp[(metlife_sftp["rut4"] != "NONE")]
metlife_sftp = metlife_sftp[(metlife_sftp["rut3"] != "NONE")]
""" metlife_sftp = metlife_sftp.drop_duplicates(
    [
        "rut1","rut2",
    ],
    keep="last",
) """
metlife_sftp.shape

(387665, 12)

In [85]:
correo = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/correo.xlsx"
dertyc = pd.read_excel(correo)
metlife_correo = dertyc.copy()

# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("-", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("–", "")
metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(" ", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.upper()
metlife_correo["rut1"] = metlife_correo["rut1"].str.strip()

metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("-", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("–", "")
metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(" ", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.upper()
metlife_correo["rut2"] = metlife_correo["rut2"].str.strip()

metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(",", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(".", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("-", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┐", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("═", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("╔", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┌", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.title()
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.strip()

metlife_correo["poliza"] = metlife_correo["poliza"].astype(str)
metlife_correo["poliza"] = metlife_correo["poliza"].str.strip()




#*******************sacar primer cero
metlife_correo.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2005110109.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2005110109.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2005110109.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["Nombre"] = metlife

(4178, 6)

In [86]:
metlife_correo["rut3"] = metlife_correo["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut4"] = metlife_correo["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo = metlife_correo[(metlife_correo["rut4"] != "NONE")]
metlife_correo = metlife_correo[(metlife_correo["rut3"] != "NONE")]

""" metlife_correo = metlife_correo.drop_duplicates(
    [
        "rut3","rut4"
    ],
    keep="last",
) """

metlife_correo.shape

(4178, 8)

In [87]:
web = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/web.xlsx"
dertyd = pd.read_excel(web)


In [88]:
dertyd.head(9)

,id,ciudad,codigo_carga,codigo_postal,comuna,convenio,convenio_plan,direccion,fecha_bloqueo,fecha_modificacion,...,Nombre,poliza,p_relacion,rut1,rut2,Titular carga,p_sexo,procesado,rut_empresa,version
0,151862,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,TAMARA CICYLIA PLAZA GUTIERREZ,340022759,1,195289484,195289484,si,2,0,651144221,1
1,151864,Chillan,0,NaN,Chillan,77,SIN PLAN,Villa Islas del Sur Pj Caulahue Casa 1362,0,2023-01-04,...,MILTON MARTINEZ VALENCIA,340013766,1,133772049,133772049,si,1,0,968026909,1
2,151869,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,PAULINA ROBERTSON MENGE,340022737,1,127222975,127222975,si,2,0,651144221,1
3,151870,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,EDUARDO ANDRES ALARCON CORTES,340022997,1,150136849,150136849,si,1,0,763558045,1
4,151871,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,BORIS ALEXANDER MAUREIRA NEIRA,340023069,1,174968683,174968683,si,1,0,764476719,1
5,151872,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,JAVIERA TERESA MADRIAGA RIVERA,340023148,1,197870648,197870648,si,2,0,775759844,1
6,151878,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,ALBERT HERRERA SANHUEZA,340023083,1,152045492,152045492,si,1,0,995754703,1
7,151881,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,LUZ MARIA LLANOS VENEGAS,340023228,1,17353857K,17353857K,si,1,0,885129005,1
8,151883,SANTIAGO,0,NaN,QUILICURA,77,SIN PLAN,PJE VOLCAN SAN PABLO 0388 SAN ENRIQUE,0,2023-01-04,...,FRANCESCA CELIS ALVEAL,340005243,1,16800656K,16800656K,si,2,0,760388734,1


In [89]:
metlife_web = dertyd.copy()

In [90]:
metlife_web.head()

,id,ciudad,codigo_carga,codigo_postal,comuna,convenio,convenio_plan,direccion,fecha_bloqueo,fecha_modificacion,...,Nombre,poliza,p_relacion,rut1,rut2,Titular carga,p_sexo,procesado,rut_empresa,version
0,151862,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,TAMARA CICYLIA PLAZA GUTIERREZ,340022759,1,195289484,195289484,si,2,0,651144221,1
1,151864,Chillan,0,NaN,Chillan,77,SIN PLAN,Villa Islas del Sur Pj Caulahue Casa 1362,0,2023-01-04,...,MILTON MARTINEZ VALENCIA,340013766,1,133772049,133772049,si,1,0,968026909,1
2,151869,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,PAULINA ROBERTSON MENGE,340022737,1,127222975,127222975,si,2,0,651144221,1
3,151870,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,EDUARDO ANDRES ALARCON CORTES,340022997,1,150136849,150136849,si,1,0,763558045,1
4,151871,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-01-04,...,BORIS ALEXANDER MAUREIRA NEIRA,340023069,1,174968683,174968683,si,1,0,764476719,1


In [91]:


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_web["rut1"] = metlife_web["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut1"] = metlife_web["rut1"].str.replace("-", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace("–", "")
metlife_web["rut1"] = metlife_web["rut1"].astype(str)
metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace(" ", "")
metlife_web["rut1"] = metlife_web["rut1"].str.upper()
metlife_web["rut1"] = metlife_web["rut1"].str.strip()

metlife_web["rut2"] = metlife_web["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].str.replace("-", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace("–", "")
metlife_web["rut2"] = metlife_web["rut2"].astype(str)
metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace(" ", "")
metlife_web["rut2"] = metlife_web["rut2"].str.upper()
metlife_web["rut2"] = metlife_web["rut2"].str.strip()

metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(",", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(".", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("-", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┐", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("═", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("╔", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┌", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.title()
metlife_web["Nombre"] = metlife_web["Nombre"].str.strip()

metlife_web["poliza"] = metlife_web["poliza"].astype(str)
metlife_web["poliza"] = metlife_web["poliza"].str.strip()

metlife_web["rut3"] = metlife_web["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut4"] = metlife_web["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web = metlife_web[(metlife_web["rut4"] != "NONE")]
metlife_web = metlife_web[(metlife_web["rut3"] != "NONE")]
metlife_web.shape

""" metlife_web = metlife_web.drop_duplicates(
    [
        "rut3","rut4"
    ],
    keep="last",
) """


#*******************sacar primer cero
metlife_web.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\533185728.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\533185728.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\533185728.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["Nombre"] = metlife_web["Nombre"].str

(4715, 41)

In [92]:
subir_correo = pd.DataFrame()
subir_correo["poliza_rut"] = metlife_correo["poliza"]
subir_correo["rut_rut"] = metlife_correo["rut1"]
subir_correo["nombre_rut"] = metlife_correo["Nombre"]
subir_correo["mes_rut"] = str(mes)
subir_correo["año_rut"] = str(año)
#subir_correo["fecha_rut"] = metlife_correo["fecha"]
subir_correo["Llave"] = metlife_correo["poliza"] + metlife_correo["rut1"] + str(mes) + str(año)
subir_correo["desde_rut"] = pd.to_datetime(metlife_correo["desde"], yearfirst=True)
subir_correo["hasta_rut"] = pd.to_datetime(metlife_correo["hasta"], yearfirst=True)

subir_sql = pd.DataFrame()
subir_sql["poliza_rut"] = metlife_sftp["poliza"]
subir_sql["rut_rut"] = metlife_sftp["rut1"]
subir_sql["nombre_rut"] = metlife_sftp["Nombre"]
subir_sql["mes_rut"] = str(mes)
subir_sql["año_rut"] = str(año)
#subir_sql["fecha_rut"] = metlife_sftp["fecha"]
subir_sql["Llave"] = metlife_sftp["poliza"] + metlife_sftp["rut1"] + str(mes) + str(año)
subir_sql["desde_rut"] = pd.to_datetime(metlife_sftp["desde"], yearfirst=True)
subir_sql["hasta_rut"] = pd.to_datetime(metlife_sftp["hasta"], yearfirst=True)

subir_web = pd.DataFrame()
subir_web["poliza_rut"] = metlife_web["poliza"]
subir_web["rut_rut"] = metlife_web["rut1"]
subir_web["nombre_rut"] = metlife_web["Nombre"]
subir_web["mes_rut"] = str(mes)
subir_web["año_rut"] = str(año)
#subir_web["fecha_rut"] = metlife_web["fecha"]
subir_web["Llave"] = metlife_web["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_web["desde_rut"] = pd.to_datetime(metlife_web["desde"], yearfirst=True)
subir_web["hasta_rut"] = pd.to_datetime(metlife_web["hasta"], yearfirst=True)

mes_metlife = pd.DataFrame()
mes_metlife = pd.concat(
    [
        subir_web,
        subir_sql,
        subir_correo,
        
    ],
    axis=0,
)
mes_metlife["fecha_rut"] = pd.to_datetime(fecha, yearfirst=True)

mes_metlife = mes_metlife.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)
mes_metlife.head()

,poliza_rut,rut_rut,nombre_rut,mes_rut,año_rut,Llave,desde_rut,hasta_rut,fecha_rut
68,340023027,178055399,Carlos Humberto Araya Jofre,1,2023,34002302717805539912023,1970-01-01 00:00:00.020230103,1970-01-01,2023-01-01
119,340002436,186177908,Nicolas Cisternas Moreno,1,2023,34000243618617790812023,1970-01-01 00:00:00.020230103,1970-01-01,2023-01-01
126,340022737,105796234,Pamela Alejandr Tillemann Munoz,1,2023,34002273710579623412023,1970-01-01 00:00:00.020230103,1970-01-01,2023-01-01
131,340023173,168497261,Pablo Alberto Fredes Fredes,1,2023,34002317316849726112023,1970-01-01 00:00:00.020230104,1970-01-01,2023-01-01
132,340004285,108120045,Rodrigo Alejand Briones Urrutia,1,2023,34000428510812004512023,1970-01-01 00:00:00.020230103,1970-01-01,2023-01-01


In [93]:
mes_metlife.shape

(363711, 9)

In [94]:
mes_metlife.to_sql(
    name="rut_unico_tiempo",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

363711

In [95]:
mes_metlife = mes_metlife.drop_duplicates(
    [
        "rut_rut"
    ],
    keep="last",
)
mes_metlife.shape

(279015, 9)

In [ ]:
subir_unicos = pd.DataFrame()
subir_unicos["Rut_T"] = mes_metlife["rut_rut"]
subir_unicos["Rut_C"] = mes_metlife["rut_rut"]
subir_unicos["Nombre"] = mes_metlife["nombre_rut"]
subir_unicos["Llave"] = mes_metlife["rut_rut"] +  mes_metlife["rut_rut"]

subir_unicos["esta"] =  subir_unicos["Rut_T"].isin(rut_unicos["Rut_T"])
subir_unicos = subir_unicos[(subir_unicos["esta"] == False)]

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]

In [ ]:
subir_unicos.to_sql(
    name="rut_unicos",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)